In [1]:
!pip install pyyaml==5.1

     |████████████████████████████████| 274 kB 5.7 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=bb538fbb21971ea18dcc4e691302c6c3e1ddb5d2446f839e5d164e445016750d
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-dydicq61
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-dydicq61
     |████████████████████████████████| 49 kB 3.0 MB/s 
     |████████████████████████████████| 74 kB 3.3 MB/s 
     |████████████████████████████████| 145 kB 43.9 MB/s 
     |████████████████████████████████| 130 kB 67.7 MB/s 
     |████████████████████████████████| 748 kB 55.2 MB/s 
     |████████████████████████████████| 743 kB 38.4 MB/s 
     |████████████████████████████████| 112 kB 71.7 MB/s 
  Created wheel for detectron2: filename=detectron2-0.6-cp37-cp37m-linux_x86_64.whl size=5606573 sha256=9b1d1dc1f67d6f0923f78080dad04553c4e716227d8d49ae53d848ddfc7d6ca5
  Stored in directory: /tmp/pip-ephem-wheel-cache-8njw66nd/wheels/07/dc/32/0322cb484dbefab8b9366bfedbaff5060ac7d149d69c27ca5d
  Created wheel for fvcore: filename=fvcore-0.1.5.post20211023-py3-none-any.whl size=60947 sha256=fed

In [3]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")

1.9.0+cu111 True


In [4]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("askui_train", {}, "/content/drive/MyDrive/Task 22.10/new_data(split).zip (Unzipped Files)/new_data(split)/askui.train.json", "/content/drive/MyDrive/Task 22.10/new_data(split).zip (Unzipped Files)/new_data(split)/train")
register_coco_instances("askui_test", {}, "/content/drive/MyDrive/Task 22.10/new_data(split).zip (Unzipped Files)/new_data(split)/askui.test.json", "/content/drive/MyDrive/Task 22.10/new_data(split).zip (Unzipped Files)/new_data(split)/test")

In [7]:
askui_train_metadata = MetadataCatalog.get("askui_train")
dataset_dicts = DatasetCatalog.get("askui_train")

[11/01 11:03:08 d2.data.datasets.coco]: Loading /content/drive/MyDrive/Task 22.10/new_data(split).zip (Unzipped Files)/new_data(split)/askui.train.json takes 1.85 seconds.
[11/01 11:03:08 d2.data.datasets.coco]: Loaded 920 images in COCO format from /content/drive/MyDrive/Task 22.10/new_data(split).zip (Unzipped Files)/new_data(split)/askui.train.json


In [8]:
askui_test_metadata = MetadataCatalog.get("askui_test")
dataset_val_dicts = DatasetCatalog.get("askui_test")

[11/01 11:03:15 d2.data.datasets.coco]: Loaded 96 images in COCO format from /content/drive/MyDrive/Task 22.10/new_data(split).zip (Unzipped Files)/new_data(split)/askui.test.json


In [9]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("askui_train",)
cfg.DATASETS.TEST = ()   
cfg.DATALOADER.NUM_WORKERS = 1
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.00025  
cfg.SOLVER.MAX_ITER = 60000
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[15, 29, 43],[62, 84, 112], [133, 142, 195], [202, 210, 280], [290, 490, 735]]
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.05, 0.1, 3]]
cfg.SOLVER.STEPS = []        
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 64

In [10]:
cfg.MODEL.WEIGHTS =  "/content/drive/MyDrive/Task 26.10/output.zip (Unzipped Files)/output/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   
predictor = DefaultPredictor(cfg)

In [11]:
from detectron2.utils.visualizer import ColorMode

In [12]:
for d in dataset_val_dicts:
  #if d["file_name"] == "/content/drive/MyDrive/Task 07.10/askui_dataset_06092021.zip (Unzipped Files)/data_splitted_equal_distribution/train/tk001.png":
  print(d["file_name"])
  img = cv2.imread(d["file_name"])
  visualizer = Visualizer(img[:, :, :], metadata=askui_test_metadata, scale=0.7)
  vis = visualizer.draw_dataset_dict(d)
  cv2_imshow(vis.get_image()[:, :, :])

Output hidden; open in https://colab.research.google.com to view.

In [14]:
for d in dataset_val_dicts:
  #if d["file_name"] == "/content/drive/MyDrive/Task 07.10/askui_dataset_06092021.zip (Unzipped Files)/data_splitted_equal_distribution/train/tk001.png":
  print(d["file_name"])
  im = cv2.imread(d["file_name"])
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=askui_test_metadata,
                scale=0.7,  
                instance_mode=ColorMode.IMAGE_BW 
  )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.

In [15]:
from detectron2.structures.instances import Instances
from detectron2.structures import Boxes

In [17]:
def bbox_iou(box1, box2):
    """
    Returns the IoU of two bounding boxes 
    得到bbox的坐标
    """
    # Get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1[0], box1[1], box1[2], box1[3]
    b2_x1, b2_y1, b2_x2, b2_y2 = box2[0], box2[1], box2[2], box2[3]

    # get the corrdinates of the intersection rectangle
    inter_rect_x1 = torch.max(b1_x1, b2_x1)
    inter_rect_y1 = torch.max(b1_y1, b2_y1)
    inter_rect_x2 = torch.min(b1_x2, b2_x2)
    inter_rect_y2 = torch.min(b1_y2, b2_y2)

    # Intersection area
    if torch.cuda.is_available():
        inter_area = torch.max(inter_rect_x2 - inter_rect_x1 + 1, torch.zeros(inter_rect_x2.shape).cuda()) * torch.max(
            inter_rect_y2 - inter_rect_y1 + 1, torch.zeros(inter_rect_x2.shape).cuda())
    else:
        inter_area = torch.max(inter_rect_x2 - inter_rect_x1 + 1, torch.zeros(inter_rect_x2.shape)) * torch.max(
            inter_rect_y2 - inter_rect_y1 + 1, torch.zeros(inter_rect_x2.shape))

    # Union Area
    b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)

    iou = inter_area / (b1_area + b2_area - inter_area)

    return iou

In [18]:
for d in dataset_val_dicts:

  #if d["file_name"] == "/content/drive/MyDrive/Task 07.10/askui_dataset_06092021.zip (Unzipped Files)/data_splitted_equal_distribution/train/tk001.png":
  print(d["file_name"])
  im = cv2.imread(d["file_name"])
  outputs = predictor(im)
  cla = outputs["instances"].pred_classes # pytorch tensor
  sco = outputs["instances"].scores
  box = outputs["instances"].pred_boxes.tensor # pytorch tensor
  ann = d["annotations"]
  for obj in ann:
    b = obj["bbox"]
    bbox = [b[0], b[1], b[0] + b[2], b[1] + b[3]]
    class_id = obj["category_id"]
    bbox = torch.tensor(bbox)
    nu = -10
    for i in range(box.size()[0]):
      io = bbox_iou(bbox, box[i])
      if io >= 0.5 and cla[i] == class_id:
        nu = i
    if nu >= 0:
      box = torch.cat((box[:nu], box[nu+1:]),dim=0)
      sco = torch.cat((sco[:nu], sco[nu+1:]),dim=0)
      cla = torch.cat((cla[:nu], cla[nu+1:]),dim=0)
  ins = Instances(outputs["instances"].image_size)
  ins.set("pred_classes", cla)
  ins.set("scores", sco)
  ins.set("pred_boxes", Boxes(box))
  v = Visualizer(im[:, :, ::-1],
                metadata=askui_test_metadata,
                scale=0.7,  
  )
  out = v.draw_instance_predictions(ins.to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.